TODO 1: Grayscale Image dataset, Resized to 40,40
- Downsample detritus images to 20000
- Saved in /shared/images_cnn/

TODO 2: Plankton augmentation
- Concentrate on copepods + noncopoepod

In [ ]:
import cv2, os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from scivision.io import load_dataset

In [ ]:
dataPath = '/data/images/'
df_ind = pd.read_csv('/data/index.csv')
df_ind.head()

In [ ]:
df_ind['label1'].value_counts()

In [ ]:
detritus_im_list = df_ind[df_ind['label1']=='detritus']['filename'].tolist()
plankton_im_list = df_ind[df_ind['label1']=='zooplankton']['filename'].tolist()

print(len(detritus_im_list), len(plankton_im_list))

In [ ]:
def preprocess(im, size, blur):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im = cv2.resize(im,(size, size))
    im = cv2.GaussianBlur(im, (blur, blur),0)
    return im

In [ ]:
savePath = '/shared/images_cnn/detritus/'

for i in sorted(detritus_im_list)[:20000]:
    im = cv2.imread(dataPath + i)
#     print(dataPath + i)
    im = preprocess(im, 40, 5)
    cv2.imwrite(savePath+i, im)

In [ ]:
print(len(os.listdir('/shared/images_cnn/detritus/')))

In [ ]:
savePath = '/shared/images_cnn/plankton/'

for i in sorted(plankton_im_list)[:5]:
    im = cv2.imread(dataPath + i)
#     print(dataPath + i)
    im = preprocess(im, 40, 5)
    cv2.imwrite(savePath+i, im)

In [ ]:
print(len(os.listdir('/shared/images_cnn/plankton/')))

# Load Data - OpenCV

In [ ]:
# dataPath = '/data/images/'
dataPath = '/shared/images_cnn/plankton/'

for i in sorted(os.listdir(dataPath))[:5]:
    im = cv2.imread(dataPath + i)
    print(i)
    print("avg/std rgb: ", np.mean(im, axis=(0,1)), np.std(im, axis=(0,1)))
    
    im = cv2.resize(im,(224,224))
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.GaussianBlur(im, (7,7),0)
    edges = cv2.Canny(im, 50, 50)
    print(im.shape)
    
    plt.subplot(131), plt.imshow(im)
    plt.subplot(132), plt.imshow(im_gray, cmap="gray")
    plt.subplot(133),plt.imshow(edges, cmap="gray")
    plt.show()

# Load Data - Scivision
- Deduplicate and view labels & example image

In [ ]:
cat = load_dataset('https://github.com/alan-turing-institute/plankton-dsg-challenge')

ds_all = cat.plankton_multiple().to_dask()
labels = cat.labels().read()

labels_dedup = xr.Dataset.from_dataframe(
    labels
    .drop_duplicates(subset=["filename"])
    .set_index("filename")
    .sort_index()
)

ds_labelled = (
    ds_all
    .swap_dims({"concat_dim": "filename"})
    .merge(labels_dedup, join="inner")
    .swap_dims({"filename": "concat_dim"})
)

In [ ]:
one_image = ds_labelled.sel(concat_dim=1)
plt.imshow(one_image['raster'].compute().values[:,:,:])
plt.title(one_image.filename.compute().values)

In [ ]:
# get image width and length into columns
ds_labelled = ds_labelled.assign(
    image_width = ds_labelled['EXIF Image ImageWidth'].to_pandas().apply(lambda x: x.values[0]),
    image_length = ds_labelled['EXIF Image ImageLength'].to_pandas().apply(lambda x: x.values[0])
)